In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install geopandas

In [4]:
!pip install typing_extensions==4.5.0 --upgrade
!pip install openai
!pip install tiktoken
!pip install --upgrade transformers
!pip install langchain
!pip install -U langchain-openai
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.8 MB/s eta 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/dataset/madison/'

## Load dataset

In [8]:
objects = pd.read_csv(path + '/input/objects_embeddings.csv', usecols=['objectid', 'geometry', 'source', 'geom_type', 'n_tokens'])
objects.head()

,objectid,geometry,source,geom_type,n_tokens
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23


In [9]:

relations = pd.read_csv(path+'input/relations.csv', usecols=['subjectid', 'objectid', 'predicate', 'relate', 'idx'])
# relations.to_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv', index=False)
# relations = pd.read_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv')
relations.head()

,objectid,subjectid,predicate,relate,idx
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0
1,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1
2,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2
3,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3
4,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4


In [10]:
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type','n_tokens']].rename(columns={'objectid': 'subjectid',
                                                        'geometry': 'geometry_subject',
                                                        'source': 'source_subject',
                                                        'geom_type': 'geom_type_subject',
                                                          'n_tokens': 'n_tokens_subject'
                                                        }), on='subjectid', how='inner')
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type', 'n_tokens']], on='objectid', how='inner')
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24


In [11]:
relations[(relations['geom_type_subject']=='Point')&(relations['geom_type']=='LineString')].sort_values(by=['n_tokens_subject', 'n_tokens']).iloc[0]

objectid                            road_(2633805462, 3423901632)
subjectid                                         node_3752320724
predicate                                                disjoint
relate                                                  FF0FFF102
idx                                                           195
geometry_subject                           POINT (-89.456 43.061)
source_subject                                       intersection
geom_type_subject                                           Point
n_tokens_subject                                               10
geometry             LINESTRING (-89.459 43.0661, -89.459 43.066)
source                                                       link
geom_type                                              LineString
n_tokens                                                       21
Name: 1383, dtype: object

In [12]:
relate_df = relations[['geom_type_subject', 'predicate', 'geom_type']].drop_duplicates().reset_index(drop=True)
relate_df['rid'] = relate_df.index
relate_df['output'] = '(' + relate_df['geom_type_subject'] + ',' + relate_df['predicate'] + ',' + relate_df['geom_type'] + ')'
relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')
# relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')

In [13]:
relations[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString       225
                            Point            225
                  crosses   LineString       225
                            Polygon          225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    LineString       225
                  overlaps  LineString       225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Point             disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Point            225
                  touches   LineString       225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Polygon           contains  LineString       225
                            Point            225
                            Polygon          225
                  crosses   LineString       225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Polygon          225
                  overlaps  Polygon          225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    Polygon          225

In [14]:
relations[['objectid', 'predicate']].groupby(['predicate']).count()

,objectid
predicate,
contains,1125
crosses,675
disjoint,2025
equals,675
overlaps,450
touches,1800
within,1125


In [15]:
relations.shape[0]

7875

## Train / Test split

In [16]:
relations_example = relations[relations['idx']>=200].reset_index(drop=True)

relations_test = relations[relations['idx']<40].reset_index(drop=True)

relations_train = relations[(relations['idx']>=40)&(relations['idx']<200)].reset_index(drop=True)

In [17]:
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,1,"(Polygon,contains,LineString)"


In [18]:
relations_test.sort_values(by=['rid', 'idx']).head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1,"LINESTRING (-89.3554 43.1351, -89.3554 43.1357)",link,LineString,24,"LINESTRING (-89.3554 43.1357, -89.3554 43.1355)",link,LineString,24,0,"(LineString,contains,LineString)"
3,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2,"LINESTRING (-89.4524 43.0716, -89.4521 43.0716...",poi,LineString,57,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...",poi,LineString,46,0,"(LineString,contains,LineString)"
6,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3,"LINESTRING (-89.3392 43.0637, -89.3399 43.0641...",poi,LineString,134,"LINESTRING (-89.338 43.0659, -89.3378 43.0659)",link,LineString,23,0,"(LineString,contains,LineString)"
10,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4,"LINESTRING (-89.5168 43.0606, -89.5168 43.0616...",poi,LineString,57,"LINESTRING (-89.5175 43.0611, -89.5175 43.0607)",link,LineString,24,0,"(LineString,contains,LineString)"


In [19]:
relations_test[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString        40
                            Point             40
                  crosses   LineString        40
                            Polygon           40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    LineString        40
                  overlaps  LineString        40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Point             disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Point             40
                  touches   LineString        40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Polygon           contains  LineString        40
                            Point             40
                            Polygon           40
                  crosses   LineString        40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Polygon           40
                  overlaps  Polygon           40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    Polygon           40

## Prompts

In [20]:
# import tiktoken, os, openai

from google.colab import userdata

KEY = userdata.get('OPENAI_QSR')
ORG = userdata.get('OPENAI_ORG')
# openai.organization = ORG
# openai.api_key = KEY
# openai.Model.list()

In [21]:
model = 'gpt-4'
model_abbr = 'gpt4'

In [22]:
from langchain_openai import ChatOpenAI
# process.env.OPENAI_ORGANIZATION = ORG
llm = ChatOpenAI(model_name=model, temperature=0, api_key=KEY, organization=ORG)

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

### Zero-shot

#### Without dimension requirement

In [23]:
relations_test.shape

(1400, 14)

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import time

SYSTEM_PROMPT = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- Return in the format (Geometry type A, PREDICATE, geometry type B), and nothing else. Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE in your output is one of the seven predicates stated."""

USER_PROMPT = """
Geometry A: {WKT_A}
Geometry B: {WKT_B}
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)
preds = []

prompt_mode = 'zero_shot'
for idx, row in relations_test[:].iterrows():
    # prompt = PROMPT.format_prompt(WKT_A=row['geometry_subject'], WKT_B=row['geometry'])
    # print(prompt)
    # pred = llm(prompt)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)

----------------
record 49 predict: (LineString, touches, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of zero_shot, using gpt4.
----------------
record 99 predict: (Polygon, CONTAINS, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of zero_shot, using gpt4.
----------------
record 149 predict: (Polygon, disjoint, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of zero_shot, using gpt4.
----------------
record 199 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of zero_shot, using gpt4.
----------------
record 249 predict: (Polygon, contains, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of zero_shot, using gpt4.
----------------
record 299 predict: (Point, within, Polygon), actual: ('Point', 'disjoint', 'Polygon')
finish saving first

In [64]:
prompt_mode = 'zero_shot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
# res_llm = pd.DataFrame(preds)
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_zero_shot_gpt4,output_zero_shot_gpt4,rid_zero_shot_gpt4
259,node_1237990690,poi_way/665181117,30,"(Point,within,LineString)","(Point, crosses, LineString)","(Point,crosses,LineString)",-1
994,node_3362623422,"road_(3362623424, 53483466)",2,"(Point,disjoint,LineString)","(Point, crosses, LineString)","(Point,crosses,LineString)",-1
1089,node_3399246912,"road_(53673607, 53663249)",30,"(Point,within,LineString)","(Point, crosses, LineString)","(Point,crosses,LineString)",-1


In [83]:
res = pd.read_csv(path+'result/task1_%s.csv'%(prompt_mode))
res.head()

,subjectid,objectid,rid,output,text_zero_shot,output_zero_shot,rid_zero_shot
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, overlaps, LineString)","(LineString,overlaps,LineString)",14
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23


In [80]:
prompt_mode='zero_shot'
res1 = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))
res1.head()

,subjectid,objectid,rid,output,text_zero_shot_gpt4,output_zero_shot_gpt4,rid_zero_shot_gpt4
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, overlaps, LineString)","(LineString,overlaps,LineString)",14
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23


In [84]:
res.shape, res1.shape

((1400, 7), (1400, 7))

In [85]:
res1.iloc[:, 4:] = res.iloc[:, 4:].values
res1.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)

In [86]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_zero_shot_gpt4,output_zero_shot_gpt4,rid_zero_shot_gpt4,geom_type_subject,predicate,geom_type,geom_type_subject_zero_shot_gpt4,predicate_zero_shot_gpt4,geom_type_zero_shot_gpt4
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, overlaps, LineString)","(LineString,overlaps,LineString)",14,LineString,contains,LineString,LineString,overlaps,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23,Polygon,contains,LineString,Polygon,touches,LineString


In [87]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.9971428571428571, 0.9992857142857143, 0.9964285714285714)

In [88]:
res_valid = res #[res['rid_%s'%prompt_mode]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [89]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.627857,0.627857,0.627857,0.627857
macro avg,0.660310,0.610417,0.560875,1400.000000
weighted avg,0.679176,0.627857,0.576899,1400.000000


In [90]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
27,26,1.000000,1.000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
22,21,1.000000,1.000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
18,17,1.000000,1.000,1.000000,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
3,2,0.973684,0.925,0.948718,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
14,13,0.944444,0.850,0.894737,40.0,LineString,touches,Point,"(LineString,touches,Point)"
13,12,0.844444,0.950,0.894118,40.0,LineString,contains,Point,"(LineString,contains,Point)"
17,16,0.860465,0.925,0.891566,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
12,11,0.942857,0.825,0.880000,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
33,32,0.770833,0.925,0.840909,40.0,Polygon,equals,Polygon,"(Polygon,equals,Polygon)"
4,3,0.864865,0.800,0.831169,40.0,LineString,disjoint,Polygon,"(LineString,disjoint,Polygon)"


#### Include dimension requirement

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import time

prompt_mode = 'zero_shot_dim'

SYSTEM_PROMPT = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- Return in the format (Geometry type A, PREDICATE, geometry type B), and nothing else. Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE in your output is one of the seven predicates stated.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
Geometry A: {WKT_A}
Geometry B: {WKT_B}
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)
preds = []


for idx, row in relations_test[:].iterrows():
    # prompt = PROMPT.format_prompt(WKT_A=row['geometry_subject'], WKT_B=row['geometry'])
    # print(prompt)
    # pred = llm(prompt)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)

----------------
record 50 predict: (LineString, TOUCHES, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of zero_shot_dim, using gpt4.
----------------
record 100 predict: (Polygon, contains, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of zero_shot_dim, using gpt4.
----------------
record 150 predict: (Polygon, disjoint, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of zero_shot_dim, using gpt4.
----------------
record 200 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of zero_shot_dim, using gpt4.
----------------
record 250 predict: (Polygon, contains, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of zero_shot_dim, using gpt4.
----------------
record 300 predict: (Point, within, Polygon), actual: ('Point', 'disjoint', 'Polygon'

In [92]:
prompt_mode = 'zero_shot_dim'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds)
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_zero_shot_dim_gpt4,output_zero_shot_dim_gpt4,rid_zero_shot_dim_gpt4
951,poi_way/32075720,polygon_parcel_60815235222,15,"(LineString,touches,Polygon)","(LineString, overlaps, Polygon)","(LineString,overlaps,Polygon)",-1


In [93]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_zero_shot_dim_gpt4,output_zero_shot_dim_gpt4,rid_zero_shot_dim_gpt4,geom_type_subject,predicate,geom_type,geom_type_subject_zero_shot_dim_gpt4,predicate_zero_shot_dim_gpt4,geom_type_zero_shot_dim_gpt4
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, TOUCHES, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, overlaps, LineString)","(LineString,overlaps,LineString)",14,LineString,contains,LineString,LineString,overlaps,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23,Polygon,contains,LineString,Polygon,touches,LineString


In [94]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.9992857142857143, 1.0, 0.9985714285714286)

In [95]:
res_valid = res #[res['rid_%s'%prompt_mode]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [96]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.633571,0.633571,0.633571,0.633571
macro avg,0.640029,0.615972,0.566163,1400.000000
weighted avg,0.658315,0.633571,0.582339,1400.000000


In [97]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
27,26,1.000000,1.000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
22,21,1.000000,1.000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
18,17,0.975610,1.000,0.987654,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
3,2,0.975000,0.975,0.975000,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
14,13,0.904762,0.950,0.926829,40.0,LineString,touches,Point,"(LineString,touches,Point)"
13,12,0.945946,0.875,0.909091,40.0,LineString,contains,Point,"(LineString,contains,Point)"
17,16,0.878049,0.900,0.888889,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
4,3,0.918919,0.850,0.883117,40.0,LineString,disjoint,Polygon,"(LineString,disjoint,Polygon)"
12,11,0.941176,0.800,0.864865,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
34,33,1.000000,0.675,0.805970,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"


### Few-shot

In [ ]:
relations_example = relations_example[relations_example['idx']==200]

In [ ]:
relations_example['n_tokens_subject'].sum() + relations_example['n_tokens'].sum()

4689

In [99]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot'

prefix = """
You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- Return in the format (Geometry type A, PREDICATE, geometry type B), and nothing else. Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE in your output is one of the seven predicates stated.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- TRY YOUR BEST to understand how the spatial relations in the examples are determined.


[EXAMPLES]
"""

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""
def get_examples(row):
    np.random.seed()
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]
    examples = (df[['geometry_subject', 'geometry', 'output']].sample(5).rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'output': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


preds = []
for idx, row in relations_test[:].iterrows():
    examples = get_examples(row)

    PROMPT = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=PROMPT)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)

----------------
record 50 predict: (LineString,touches,LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of few_shot, using gpt4.
----------------
record 100 predict: (Polygon,touches,LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of few_shot, using gpt4.
----------------
record 150 predict: (Polygon,disjoint,LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of few_shot, using gpt4.
----------------
record 200 predict: (LineString,disjoint,Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of few_shot, using gpt4.
----------------
record 250 predict: (Polygon,contains,Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of few_shot, using gpt4.
----------------
record 300 predict: (Point,within,Polygon), actual: ('Point', 'disjoint', 'Polygon')
finish saving first 300 predictions 

In [100]:
prompt_mode = 'few_shot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds)
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_few_shot_gpt4,output_few_shot_gpt4,rid_few_shot_gpt4
118,polygon_census_550250021001,"road_(10781526611, 5035134454)",23,"(Polygon,touches,LineString)","(Polygon,within,LineString)","(Polygon,within,LineString)",-1
260,poi_way/102303042,census_550250016043,6,"(LineString,crosses,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1
368,parcel_80925113112,poi_way/118690988,23,"(Polygon,touches,LineString)","(Polygon,overlaps,LineString)","(Polygon,overlaps,LineString)",-1
482,poi_way/377060726,parcel_71007229084,6,"(LineString,crosses,Polygon)","(LineString,equals,Polygon)","(LineString,equals,Polygon)",-1
484,poi_way/377060724,parcel_71007229084,15,"(LineString,touches,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1
576,poi_way/244981734,parcel_70923174448,15,"(LineString,touches,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1
579,poi_way/244981734,parcel_70923173515,15,"(LineString,touches,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1
667,poi_way/175095307,parcel_70922117184,15,"(LineString,touches,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1
668,line_poi_way/175095307,parcel_70922117184,15,"(LineString,touches,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1
951,poi_way/32075720,polygon_parcel_60815235222,15,"(LineString,touches,Polygon)","(LineString,overlaps,Polygon)","(LineString,overlaps,Polygon)",-1


In [101]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_few_shot_gpt4,output_few_shot_gpt4,rid_few_shot_gpt4,geom_type_subject,predicate,geom_type,geom_type_subject_few_shot_gpt4,predicate_few_shot_gpt4,geom_type_few_shot_gpt4
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString,touches,LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString,contains,LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString,contains,LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString,overlaps,LineString)","(LineString,overlaps,LineString)",14,LineString,contains,LineString,LineString,overlaps,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon,disjoint,LineString)","(Polygon,disjoint,LineString)",11,Polygon,contains,LineString,Polygon,disjoint,LineString


In [102]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.9921428571428571, 1.0, 0.9992857142857143)

In [103]:
res_valid = res #[res['rid_%s'%prompt_mode]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [104]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.660714,0.660714,0.660714,0.660714
macro avg,0.666283,0.642361,0.629866,1400.000000
weighted avg,0.685319,0.660714,0.647862,1400.000000


In [105]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
27,26,1.000000,1.000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
22,21,1.000000,1.000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
18,17,0.975610,1.000,0.987654,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
3,2,0.975610,1.000,0.987654,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
34,33,0.970588,0.825,0.891892,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
17,16,0.804348,0.925,0.860465,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
14,13,0.750000,0.975,0.847826,40.0,LineString,touches,Point,"(LineString,touches,Point)"
26,25,0.966667,0.725,0.828571,40.0,Polygon,touches,Point,"(Polygon,touches,Point)"
25,24,0.709091,0.975,0.821053,40.0,Polygon,contains,Point,"(Polygon,contains,Point)"
12,11,0.772727,0.850,0.809524,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"


## Chain of Thought

#### Zero-shot-CoT

In [108]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains import LLMChain

prompt_mode = 'zero_shot_cot'

SYSTEM_PROMPT = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the [Predicate] is one of the seven predicates stated,
and the Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
Geometry A: {WKT_A}
Geometry B: {WKT_B}
"""

AI_PROMPT = """
Let's think step by step.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
        ("ai", AI_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)

preds = []
for idx, row in relations_test[:].iterrows():
    # prompt = PROMPT.format_prompt(WKT_A=row['geometry_subject'], WKT_B=row['geometry'])
    # print(prompt)
    # pred = llm(prompt)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(pred['text'])
    # print('****')

    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 50 predict: The two geometries are both LineStrings. Geometry A is a line segment that ends at the point (-89.3108, 43.0482). Geometry B is a line segment that starts at the point (-89.3108, 43.0482). 

The two line segments share a common point, which is the end point of Geometry A and the start point of Geometry B. However, they do not share any line segments. 

According to the definitions of the spatial relations, the two geometries are said to 'touch' if they have at least one point in common, but their interiors do not intersect. In this case, the two geometries share a common point, but their interiors (the parts of the line segments excluding the end points) do not intersect. Therefore, the spatial relation between the two geometries is 'touches'.

ANSWER: (LineString, TOUCHES, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of zero_shot_cot, using gpt4.
----------------
record 100 predict: First, we need 

In [125]:
relations_test.shape

(1400, 15)

In [119]:
res_llm = pd.DataFrame(preds)
res_llm.shape

(2800, 3)

In [126]:
res_llm = pd.read_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode))
res_llm.shape

(2800, 3)

In [127]:
res_llm.iloc[1]['text']

"The two geometries are both LineStrings. Geometry A is a line segment that ends at the point (-89.3386, 43.0922). Geometry B is a line segment that starts at the same point (-89.3386, 43.0922). \n\nThe two line segments share a common point, but they do not overlap each other. Therefore, the spatial relation between these two geometries is 'touches'. \n\nAccording to the DE-9IM model, 'touches' is defined as two geometries having at least one common point, but their interiors do not intersect. This is exactly the case for Geometry A and Geometry B. \n\nThe Open Geospatial Consortium (OGC) also defines 'touches' in the same way. Therefore, the spatial relation between Geometry A and Geometry B is 'touches' according to both DE-9IM and OGC.\n\nANSWER: (LineString, TOUCHES, LineString)"

In [128]:
res_llm.iloc[0]['text']

"The two LineStrings share a common point, which is (-89.3386 43.0922). However, they do not share any common segments or overlap each other. According to the DE-9IM matrix, this situation is defined as 'touches'. The 'touches' predicate is true when the intersection of the interiors of the geometries is empty but the intersection of the boundary and interior of the two geometries is not empty.\n\nANSWER: (LineString, TOUCHES, LineString)"

In [134]:
prompt_mode = 'zero_shot_cot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds).drop_duplicates(subset='idx', keep='last')
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_zero_shot_cot_gpt4,output_zero_shot_cot_gpt4,rid_zero_shot_cot_gpt4
20,poi_way/323552582,"road_(9670254706, 9327940393)",0,"(LineString,contains,LineString)","(LineString, NONE, LineString)","(LineString,none,LineString)",-1
21,poi_way/323552582,"road_(9327940393, 9670254706)",0,"(LineString,contains,LineString)","(LineString, NONE, LineString)","(LineString,none,LineString)",-1
40,poi_way/177921886,"road_(3297233870, 3297233868)",14,"(LineString,overlaps,LineString)","(LineString, TOUCHES, LineString)\nANSWER: (Li...","(LineString,touches,LineString)\nanswer:(LineS...",-1
66,census_550250111042,"road_(53461235, 3891862900)",1,"(Polygon,contains,LineString)","(Polygon, WITHIN, LineString)","(Polygon,within,LineString)",-1
107,poi_way/258408012,"road_(53319125, 1236925410)",14,"(LineString,overlaps,LineString)","(LineString, CONTAINS, LineString)\nANSWER: (L...","(LineString,contains,LineString)\nanswer:(Line...",-1
...,...,...,...,...,...,...,...
1294,census_550250029003,"road_(8412038329, 6865857313)",27,"(Polygon,crosses,LineString)","(Polygon, WITHIN, LineString)","(Polygon,within,LineString)",-1
1322,census_550250011012,"road_(4783030526, 4783030523)",27,"(Polygon,crosses,LineString)","(Polygon, WITHIN, LineString)","(Polygon,within,LineString)",-1
1325,census_550250006004,"road_(6177889980, 53400386)",27,"(Polygon,crosses,LineString)","(Polygon, WITHIN, LineString)","(Polygon,within,LineString)",-1
1330,census_550250026024,node_1234314625,25,"(Polygon,touches,Point)","(Polygon, WITHIN, Point)","(Polygon,within,Point)",-1


In [142]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

# res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
# res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
# res['geom_type_subject'] = res['geom_type_subject'].str[1:]
# res['geom_type'] = res['geom_type'].str[:-1]

res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
res[['geom_type_subject_cot_fewshot', 'predicate_cot_fewshot', 'geom_type_cot_fewshot']] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_cot_fewshot'] = res['geom_type_subject_cot_fewshot'].str[1:]
res['geom_type_cot_fewshot'] = res['geom_type_cot_fewshot'].str[:-1]
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_zero_shot_cot_gpt4,output_zero_shot_cot_gpt4,rid_zero_shot_cot_gpt4,geom_type_subject,predicate,geom_type,geom_type_subject_cot_fewshot,predicate_cot_fewshot,geom_type_cot_fewshot,geom_type_subject_zero_shot_cot_gpt4,predicate_zero_shot_cot_gpt4,geom_type_zero_shot_cot_gpt4
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, TOUCHES, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, OVERLAPS, LineString)","(LineString,overlaps,LineString)",14,LineString,contains,LineString,LineString,overlaps,LineString,LineString,overlaps,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, TOUCHES, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, TOUCHES, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, DISJOINT, LineString)","(Polygon,disjoint,LineString)",11,Polygon,contains,LineString,Polygon,disjoint,LineString,Polygon,disjoint,LineString


In [143]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]!=-1)
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]!=-1)

validity, subject_type_match, object_type_match

(0.9521428571428572, 0.9924981245311327, 0.9894973743435859)

In [147]:
res_valid = res [res['rid_%s_%s'%(prompt_mode, model_abbr)]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [148]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.603151,0.603151,0.603151,0.603151
macro avg,0.637134,0.595423,0.550138,1333.000000
weighted avg,0.641872,0.603151,0.557851,1333.000000


In [149]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
21,21,1.000000,1.000000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
26,26,1.000000,1.000000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
17,17,0.869565,1.000000,0.930233,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
33,33,0.918919,0.894737,0.906667,38.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
2,2,0.816327,1.000000,0.898876,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
11,11,0.809524,0.871795,0.839506,39.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
4,4,1.000000,0.700000,0.823529,40.0,Point,disjoint,Polygon,"(Point,disjoint,Polygon)"
16,16,0.703704,0.950000,0.808511,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
13,13,0.639344,1.000000,0.780000,39.0,LineString,touches,Point,"(LineString,touches,Point)"
3,3,0.686275,0.897436,0.777778,39.0,LineString,disjoint,Polygon,"(LineString,disjoint,Polygon)"


#### Few-shot-CoT

In [28]:
# for _, row in relations_example.iterrows():

def get_cot_example(row):
    relate = list(row['relate'])
    WKT_A = row['geometry_subject']
    WKT_B = row['geometry']
    Type_A = row['geom_type_subject']
    Type_B = row['geom_type']

    example = f"""
First, let's determine the geometry types of A and B.
Geometry A is a {Type_A}.
Geometry B is a {Type_B}.

"""
    predicate = None
    # within = False
    # contains = False

    idx = 1
    IA_IB = 'No' if row['predicate'] in ['disjoint', 'touches'] else 'Yes' #relate[0]=='F' else 'Yes'
    example += f"{idx}. Does the Interior of A intersect the Interior of B? \n{IA_IB}\n"
    idx+=1



    if IA_IB == 'No':
        BA_BB = 'No' if row['predicate']=='disjoint' else 'Yes' #relate[4]=='F' else 'Yes'
        example += f"{idx}. Does A intersect B? \n{BA_BB}\n"
        idx+=1

        if BA_BB=='No':
            predicate = 'disjoint'
        else:
            predicate = 'touches'

        example += f"\nSo {Type_A} A {predicate} {Type_B} B.\n"


    else:
        IA_IB = 'Yes' if (row['predicate'] in ['within', 'equals']) else 'No'
        example += f"{idx}. Does the intersection of A and B equal to  A? \n{IA_IB}\n"
        idx+=1

        if IA_IB=='Yes':
            contains = "No" if row['predicate']=='Within' else "Yes"
            example += f"{idx}. Does the intersection of  A and  B equal to  B? \n{contains}\n"
            idx += 1
            if contains=="No":
                predicate = 'within'
                example += f"So {Type_A} A {predicate} {Type_B} B.\n"

            else:
                predicate = 'equals'
                example += f"So {Type_A} A is {predicate} {Type_B} B.\n"


        if (IA_IB=='No'):
            IA_IB = 'Yes' if (row['predicate'] in ['contains']) else 'No'
            example += f"{idx}. Does the intersection of  A and  B equal to  B? \n{IA_IB}\n"
            idx+=1

            if IA_IB=='Yes':
                predicate = 'contains'
                example += f"So {Type_A} A {predicate} {Type_B} B.\n"
            else:
                Dim = 'No' if (row['predicate']=='crosses') else 'Yes'

                # example += f"""{idx}. Let Dim(A) be the dimension of the Interior of A,
                # Dim(B) be the dimension of the Interior of B, Dim(AB) be the dimension of
                # the Interior of the intersection of A and B. Does Dim(AB)=max(Dim(A), Dim(B))-1 hold? \n{Dim}\n"""
                example += f"""{idx}. Are A an B of the same dimension and
                  their intersection set results in a geometry different from both but of the same dimension?\n{Dim}\n"""
                idx+=1

                if Dim=='No':
                    predicate = 'crosses'
                else:
                    predicate = 'overlaps'
                example += f"So {Type_A} A {predicate} {Type_B} B.\n"


    example += f"ANSWER: ({Type_A}, {predicate}, {Type_B})"

    # print('---------')
    # print(row['predicate'], predicate)
    # print(example)

    return example


In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.prompts.prompt import PromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot_cot'

prefix = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the PREDICATE is one of the seven predicates stated,
and the Geometry Types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- MAKE SURE the same decision process as shown in the examples is used to reason about the answer.
- DO NOT ask repeated questions.

[EXAMPLES]
"""

# """You will be given the WKT format of geometries given the subject A and reference object B.
# Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
# The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

# MAKE SURE the rationale is complete.
# MAKE SURE the answer ends with the 'ANSWER: (geometry type A, predicate, geometry type B)', where the Geometry types are Point, LineString, Polygon.
# and the predicate is one of the six predicates stated. """

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""
relations_example = relations_example[relations_example['idx']==200]
def get_examples(row):
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]

    df['example'] = df.apply(lambda row: get_cot_example(row), axis=1)

    examples = (df[['geometry_subject', 'geometry', 'example']].rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'example': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


preds = []
for idx, row in relations_test[:].iterrows():
    examples = get_examples(row)

    prompt = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=prompt)
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    # # print(pred)
    # print('predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(prompt.format(WKT_A=row['geometry_subject'], WKT_B=row['geometry']))

    # print('******')
    # print(pred['text'])

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 50 predict: (LineString, touches, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of few_shot_cot, using gpt4.
----------------
record 100 predict: (Polygon, touches, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of few_shot_cot, using gpt4.
----------------
record 150 predict: (Polygon, disjoint, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of few_shot_cot, using gpt4.
----------------
record 200 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of few_shot_cot, using gpt4.
----------------
record 250 predict: (Polygon, contains, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of few_shot_cot, using gpt4.
----------------
record 300 predict: (Point, within, Polygon), actual: ('Point', 'disjoint', 'Polygon')
fini

In [30]:

prompt_mode = 'few_shot_cot'
preds = pd.read_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode)).to_dict('records')
start_idx = len(preds)
start_idx

800

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
import time

prompt_mode = 'few_shot_cot'

prefix = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the PREDICATE is one of the seven predicates stated,
and the Geometry Types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- MAKE SURE the same decision process as shown in the examples is used to reason about the answer.
- DO NOT ask repeated questions.

[EXAMPLES]
"""

# """You will be given the WKT format of geometries given the subject A and reference object B.
# Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
# The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

# MAKE SURE the rationale is complete.
# MAKE SURE the answer ends with the 'ANSWER: (geometry type A, predicate, geometry type B)', where the Geometry types are Point, LineString, Polygon.
# and the predicate is one of the six predicates stated. """

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""

relations_example = relations_example[relations_example['idx']==200]
def get_examples(row):
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]

    df['example'] = df.apply(lambda row: get_cot_example(row), axis=1)

    if df.shape[0]>5:
        df = df.sample(5)

    examples = (df[['geometry_subject', 'geometry', 'example']].rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'example': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


# preds = []
for idx, row in relations_test.iloc[start_idx:].iterrows():
    examples = get_examples(row)

    prompt = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=prompt)
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    # # print(pred)
    # print('predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(prompt.format(WKT_A=row['geometry_subject'], WKT_B=row['geometry']))

    # print('******')
    # print(pred['text'])

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 850 predict: (LineString, within, LineString), actual: ('LineString', 'overlaps', 'LineString')
finish saving first 850 predictions of few_shot_cot, using gpt4.
----------------
record 900 predict: (Polygon, disjoint, Polygon), actual: ('Polygon', 'disjoint', 'Polygon')
finish saving first 900 predictions of few_shot_cot, using gpt4.
----------------
record 950 predict: (LineString, touches, Polygon), actual: ('LineString', 'touches', 'Polygon')
finish saving first 950 predictions of few_shot_cot, using gpt4.
----------------
record 1000 predict: (Point, within, Polygon), actual: ('Point', 'touches', 'Polygon')
finish saving first 1000 predictions of few_shot_cot, using gpt4.
----------------
record 1050 predict: (Polygon, disjoint, Point), actual: ('Polygon', 'disjoint', 'Point')
finish saving first 1050 predictions of few_shot_cot, using gpt4.
----------------
record 1100 predict: (Polygon, contains, Polygon), actual: ('Polygon', 'contains', 'Polygon')
finish 

In [32]:
prompt_mode = 'few_shot_cot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds).drop_duplicates(subset='idx', keep='last')
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_few_shot_cot_gpt4,output_few_shot_cot_gpt4,rid_few_shot_cot_gpt4
482,poi_way/377060726,parcel_71007229084,6,"(LineString,crosses,Polygon)","(LineString, equals, Polygon)","(LineString,equals,Polygon)",-1
484,poi_way/377060724,parcel_71007229084,15,"(LineString,touches,Polygon)","(LineString, equals, Polygon)","(LineString,equals,Polygon)",-1


In [33]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

# res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
# res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
# res['geom_type_subject'] = res['geom_type_subject'].str[1:]
# res['geom_type'] = res['geom_type'].str[:-1]

res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
res[['geom_type_subject_cot_fewshot', 'predicate_cot_fewshot', 'geom_type_cot_fewshot']] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_cot_fewshot'] = res['geom_type_subject_cot_fewshot'].str[1:]
res['geom_type_cot_fewshot'] = res['geom_type_cot_fewshot'].str[:-1]
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_few_shot_cot_gpt4,output_few_shot_cot_gpt4,rid_few_shot_cot_gpt4,geom_type_subject,predicate,geom_type,geom_type_subject_cot_fewshot,predicate_cot_fewshot,geom_type_cot_fewshot,geom_type_subject_few_shot_cot_gpt4,predicate_few_shot_cot_gpt4,geom_type_few_shot_cot_gpt4
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString,LineString,contains,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString,LineString,contains,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString,LineString,contains,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, disjoint, LineString)","(Polygon,disjoint,LineString)",11,Polygon,contains,LineString,Polygon,disjoint,LineString,Polygon,disjoint,LineString


In [34]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]!=-1)
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]!=-1)

validity, subject_type_match, object_type_match

(0.9985714285714286, 0.9992846924177397, 0.9992846924177397)

In [35]:
res_valid = res [res['rid_%s_%s'%(prompt_mode, model_abbr)]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [36]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.626609,0.626609,0.626609,0.626609
macro avg,0.645558,0.626227,0.608315,1398.000000
weighted avg,0.645802,0.626609,0.608613,1398.000000


In [37]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
26,26,1.000000,1.000000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
21,21,1.000000,1.000000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
2,2,1.000000,0.950000,0.974359,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
17,17,0.930233,1.000000,0.963855,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
16,16,0.945946,0.875000,0.909091,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
33,33,0.921053,0.875000,0.897436,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
3,3,0.894737,0.850000,0.871795,40.0,LineString,disjoint,Polygon,"(LineString,disjoint,Polygon)"
11,11,0.760870,0.875000,0.813953,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
24,24,0.692308,0.900000,0.782609,40.0,Polygon,contains,Point,"(Polygon,contains,Point)"
22,22,0.828571,0.725000,0.773333,40.0,LineString,equals,LineString,"(LineString,equals,LineString)"
